<a href="https://colab.research.google.com/github/Sidd1609/Titanic_KNN/blob/master/Titanic_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import csv

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [6]:
from google.colab import files
uploaded=files.upload()

Saving gender_submission.csv to gender_submission.csv


In [0]:
df_train = pd.read_csv('train.csv')
df_train.head()
df_train.shape
df_train.dtypes

In [0]:
df_train = df_train.drop('Name', axis=1,)
df_train = df_train.drop('Ticket', axis=1,)
df_train = df_train.drop('Fare', axis=1,)
df_train = df_train.drop('Cabin', axis=1,)
df_train.dtypes

In [0]:
df_train['Family'] = df_train['SibSp'] + df_train['Parch'] + 1
df_train = df_train.drop('SibSp', axis=1,)
df_train = df_train.drop('Parch', axis=1,)
df_train["Age"].describe()

In [0]:
feat_list = list(df_train.columns.values)

for feat in feat_list:
    print (feat,": ",sum(pd.isnull(df_train[feat])))
df_train["Age"] = df_train["Age"].fillna(df_train["Age"].median())
df_train["Embarked"].mode()
df_train["Embarked"] = df_train["Embarked"].fillna("S")
feat_list = list(df_train.columns.values)

for feat in feat_list:
    print (feat,": ",sum(pd.isnull(df_train[feat])))

In [0]:
df_train["Age"].describe()
df_train["Adult"] = 0
df_train["Adult"][df_train["Age"] >= 18] = 1
print ("Passengers more than 18 yrs old: ",str(len(df_train[df_train["Age"] >= 18])))
print ("Number of Adults: ",str(len(df_train[df_train["Adult"] >= 1])))

In [0]:
df_train = df_train.drop('Age', axis=1,)
df_train.head()
def make_pivot (param1, param2):
    df_slice = df_train[[param1, param2, 'PassengerId']]
    slice_pivot = df_slice.pivot_table(index=[param1], columns=[param2],aggfunc=np.size, fill_value=0)
    
    p_chart = slice_pivot.plot.bar()
    for p in p_chart.patches:
        p_chart.annotate(str(p.get_height()), (p.get_x() * 1.05, p.get_height() * 1.01))
    
    return slice_pivot
    return p_chart
make_pivot("Survived","Pclass")
make_pivot("Survived","Sex")
make_pivot("Survived","Embarked")
make_pivot("Survived","Family")
make_pivot("Survived","Adult")
df1 = df_train.filter(['Pclass','Sex','Embarked','Family','Adult'], axis=1)
X = df1
df2 = df_train['Survived']
y = df2
X["Embarked"].unique()

In [0]:
X["Embarked"][df_train["Embarked"] == "S"] = 1
X["Embarked"][df_train["Embarked"] == "C"] = 2
X["Embarked"][df_train["Embarked"] == "Q"] = 3
X["Sex"][df_train["Sex"] == "male"] = 1
X["Sex"][df_train["Sex"] == "female"] = 2
test = SelectKBest(f_classif, k='all')
test_fit = test.fit(X, y)
feat_score = test_fit.scores_.round(3)
p_values = -np.log10(test_fit.pvalues_).round(3)
feature_list = list(X.columns.values)
selected_features = test.get_support([test_fit])
selected_features
temp_list = [ ]

for i in selected_features:
    temp_list.append({'Feature':feature_list[i], 'P_Value':p_values[i], 'Score': feat_score[i]  })
    
feat_select = pd.DataFrame(temp_list)
feat_select = feat_select.sort_values(by='Score', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')
feat_select = feat_select.set_index('Feature')
feat_select
ax = feat_select[['P_Value','Score']].plot(kind='bar', title ="Scores and P Values",  legend=True, fontsize=12)
ax.set_xlabel("Feature", fontsize=12)
ax.set_ylabel("Scores and P Values", fontsize=12)
plt.show()
X = X.drop('Embarked', axis=1,)
X = X.drop('Family', axis=1,)
X.head()


In [0]:
features_train, features_test, labels_train, labels_test = train_test_split(X, y, test_size=0.3, random_state=42)
features_train.shape
features_test.shape
labels_train.shape
labels_test.shape

In [0]:
#KNN
knn = KNeighborsClassifier( )
k_range = list(range(1,10))
weights_options = ['uniform','distance']
k_grid = dict(n_neighbors=k_range, weights = weights_options)
grid = GridSearchCV(knn, k_grid, cv=10, scoring = 'precision')
grid.fit(features_train, labels_train)
grid.grid_scores_
print ("Best Score: ",str(grid.best_score_))
print ("Best Parameters: ",str(grid.best_params_))
print ("Best Estimators: ",str(grid.best_estimator_))

In [0]:
#Accuracy
label_pred = grid.predict(features_test)
acc_clf = metrics.accuracy_score(labels_test,label_pred)
print ("classifier's accuracy: ",str(acc_clf) )
scr_clf = precision_recall_fscore_support(labels_test,label_pred, average='weighted')

print ("classifier's precision: ",str(scr_clf[0])) 
print ("classifier's recall: ",str(scr_clf[1])) 
print ("classifier's fbeta_score: ",str(scr_clf[2])) 
df_test = pd.read_csv('test.csv')

In [0]:
df_test.dtypes
test_set = df_test[['Pclass', 'Sex', 'Age']].copy()
test_set.columns.values
test_list = list(test_set.columns.values)

for test_item in test_list:
    print (test_item,": ",sum(pd.isnull(test_set[test_item])))
test_set["Age"] = test_set["Age"].fillna(test_set["Age"].median())
test_set["Adult"] = 0
test_set["Adult"][test_set["Age"] >= 18] = 1
test_set = test_set.drop('Age', axis=1,)
test_set["Sex"][test_set["Sex"] == "male"] = 1
test_set["Sex"][test_set["Sex"] == "female"] = 2

In [0]:
test_set.head()
final_pred = grid.predict(test_set)
final_pred
len(final_pred)
df_test['Survived'] = pd.Series(final_pred, index=df_test.index)
df_test.head()

In [0]:
final_df = df_test.filter(['PassengerId','Survived'], axis=1)
final_df.shape